# My First Neuron Simulator 𖡬

![Johannes Blenke](https://images.unsplash.com/photo-1662242742653-13f0f76de431?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1740&q=80)

Like how libraries provide packaged functionality, we can create a **Class** that holds *attributes*  (or *properties*) and *methods* (and *functions*). 

This helps **abstract** a lot of detail by **ENCAPSULATING** *implementation* details and allowing us to re-use blocks of related code easily (i.e. collated pieces of logic forming a concept / object). 

You should aim to abstract away as much detail as necessary, but not more. 

> The process of re-visiting old code and changing it (*hopefully an improvement!*) is known as **refactorisation** (noun) or **refactoring** (verb).

You can define an object by creating a **CLASS** definition using the syntax

```
class <MyClassName>:
  """some comment about the class"""

  def __init__(self):
    <things to do when object is created>

  def <other class method>(self, a):
    """a short comment about the method"""
    self.a  = a # this class now has property a!
```

> Note how the first parameter for a class method must be `self`. This provides the method with access to *other* class methods and properties. It's also a way to store arguments provided to methods as object properties. 

> `__init__` is a special method name (hence it starts *and* ends with `__`). It is called when a class is *initialised* or created (also known as **instantiation**) using `obj = MyClassName()`). `obj` is the **instantiated OBJECT** of the **class**. Hence, "Object Orientated Programming", or OOP.

> Classes define the attributes and methods, Objects are what you play around with when you run your program.

> There are a number of other special methods that classes have that can be overwritten!

In [ ]:
%run "2 - synapses.ipynb"

## Encapsulate the neuron logic so far in a single entity 🎁

In [ ]:
import math

# create 'constants' with uppercase variable names
DEFAULT_V_REST = -65  # resting membrane potential (mV)
DEFAULT_C_M = 1       # membrane capacitance (nF)
DEFAULT_R_M = 100     # membrane resistance (MOhm)

# define a 'point' neuron's core properties
class MyFirstNeuron:
    """ A point neuron that can receive spike times for GABAA and AMPA synapses, 
    as well as constant external current. Note this only models subthreshold 
    membrane potentials (no action potentials are generated).

    Note: although a point neuron, the length and radius of the neuron must be
    provided (in um).
    """
    def __init__(self, length, radius, name="my first neuron", 
               R_m=DEFAULT_R_M, C_m=DEFAULT_C_M, V_0=DEFAULT_V_REST):
        print(f"creating neuron {name}")
        # initialise variables
        self.name = name          # name now belongs, and is a property of "MyFirstNeuron"
        self.length = length*1e-5 # um to dm
        self.r = radius*1e-5      # um to dm
        self.R_m = R_m
        self.C_m = C_m
        self.V_rest = V_0
        self.I_e = 0

        # surface area of electrode placement (cm2)
        self.A = (np.pi * self.length * self.r)*100              
        # volume of sphere (dm3)     
        self.volume = 4/3 * np.pi * (self.length/2) * self.r**2      
        # surface area of open cylinder (cm2)
        self.SA = (self.length*np.pi*self.r + 2*np.pi*self.r**2)*100  

        # create lists, even if they start empty
        self.V_t = []               # we are going to record the voltage
        self.inh_spks = []            # list for spikes that arrive at GABA synapses
        self.exc_spks = []            # list for spikes that arrive at AMPA synapses

        # initialise variables for a simulation
        self.reinit()

    def set_inh_input(self, inh_spks):
        self.inh_spks = inh_spks

    def set_exc_input(self, exc_spks):
        self.exc_spks = exc_spks

    def set_external_input(self, I_e):
        # current is in μA
        self.I_e = I_e

    def _I_syn(self, t, V_t):
        """ Returns the sum of all synaptic currents.
        Note that the method starts with an underscore "_" to indicate that
        it should be for internal (to the class) use only!
        """
        return I_GABA(t, V_t, self.inh_spks) + I_AMPA(t, V_t, self.exc_spks)

    def step(self, t, dt):
        V_t = self.V_t[-1] # get last value
        I_leak = (V_t - self.V_rest)/self.R_m
        dV = (-I_leak - self._I_syn(t, V_t) + self.I_e*1e-5/self.A) * dt/self.C_m
        self.V_t.append(V_t + dV)

    def reinit(self, V_0=None):
        """Re-initialise the membrane potential list, starting at V_0 (or V_rest 
        if no argument is provided).
        """
        if V_0 is None:   # None is a special value that provides a good placeholder
            V_0 = self.V_rest # use original value
        self.V_t = [V_0]

> Note that methods that start with a single underscore *discourage* use    outside the class. E.g. `_I_syn`

> Double underscore makes the method private (difficult to 'see' outside the  class and child classes).

## Add some logic that allows us to 'step through' time. 🚶

Objects in the simulator move through time through their `step` method.

In [ ]:
class Simulator:
    """ A Simulator can be used to step through time by calling each of its
    encapsulated neuron's ``MyFirstNeuron.step`` method.
    The duration and the time step can be chosen when call ``run`` or 
    ``continuefor`` (which does does initialise objects)
    The time is recorded in the ``t_list`` variable for retrieval later.

    Note that only one Simulator ever exists.
    We call this a Singleton class because efforts to construct a new object 
    instead retrieve the existing object and so there only exists a single object.
    Here, this is implemented here by 
    - overriding the __new__ method (takes in `cls` - shorthand for 'class' - 
    instead of `self`)
    - having a _sim class variable (it is at the top of the class definition)
    other approaches can be found
    https://python-3-patterns-idioms-test.readthedocs.io/en/latest/Singleton.html
    """
    _sim = None

    def __new__(cls):
        if not cls._sim:
            print("new Simulator")
            cls._sim = super(Simulator, cls).__new__(cls)
            cls._sim.neurons = []
        else:
            print("existing Simulator")
        return cls._sim
  
    def __init__(self):
        self.init()
  
    def init(self):
        self.t_list = [0]
        # reinitialise all neurons (for repeated simulations)
        for neuron in self.neurons:
            neuron.reinit()

    def add_neuron(self, neuron):
        # ensure only neurons with unique names are in the list
        for nrn in self.neurons:
            if neuron.name == nrn.name:  # found a previous version
                sim.neurons.remove(nrn)    # remove from simulator
                del nrn                    # delete object
        self.neurons.append(neuron)

    def _run(self, duration, time_step):
        """private method to run the simulation from t_list[-1] until duration.
        """
        print()
        start = self.t_list[-1]
        stop = self.t_list[-1]+duration+time_step
        for t in np.arange(start, stop, time_step):
            self.t_list.append(t)
            for neuron in self.neurons:
                neuron.step(t, time_step)
            # if i%log_every==0:
            #   print("#"*int(100*i/num_iter) + f"> {t} ms", end='\r')
        print()

    def run(self, duration=100, time_step=0.1):
        """Run a simulation over time, calling the step method of each neuron in the
        neurons list. 

        Values at t_0 are defined in all contained objects' reinit() methods
        The first time step is therefore t_1 = dt.
        """
        num_iter = int(duration/time_step)
        print(f"running a simulation for {duration} ms "
              f"using a dt of {time_step} ms."
              f"\nThere will be {num_iter} iterations for each of "
              f"{len(self.neurons)} neurons")
        self.init()
        self._run(duration, time_step)
        print("done!")
  
    def continuefor(self, duration=100, time_step=0.1):
        num_iter = int(duration/time_step)
        print(f"continuing a simulation from {self.t_list[-1]} ms "
              f"for {duration} ms " 
              f"using a dt of {time_step} ms."
              f"\nThere will be {num_iter} iterations for each of "
              f"{len(self.neurons)} neurons")
        self._run(duration, time_step)
        print("done!")

    def clean(self):
        while len(self.neurons)>0:
            neuron = self.neurons.pop()
            del neuron
        self.init()
        return self

## Simulate a Neuron! 🎉

In [ ]:
import numpy as np
T = 1000 # duration (ms)
dt = 0.1 # time step (ms)

# create a simulator
sim = Simulator()

# create a neuron with input
nrn = MyFirstNeuron(23, 11, name='my 1st neuron!')
nrn.set_inh_input(np.random.randint(0, T, size=10))
nrn.set_exc_input(np.random.randint(0, T, size=10))

# TODO: create ANOTHER neuron with external input
# ...

# adding all neurons we want to update
sim.add_neuron(nrn)

# run a simulation of a neuron
sim.run(T, dt)

# visualise
import matplotlib.pyplot as plt

# create a figure with an axis for plotting
fig, ax = plt.subplots()

# plot voltage over iterations
ax.plot(sim.t_list, nrn.V_t)

plt.show()

## **Tasks** ✏️



**Task 1:** Run the cell below with different time steps. 

- How does this affect the voltage trace?

**Task 2:** Create another neuron with additional external input and compare the traces

- So the same excitatory and inhibitory input as "my 1st neuron!" but **also** with external input.
- We will use this neuron in future traces

In [ ]:
#@title Task 2 Solution { display-mode: "form" }
T = 1000 # duration (ms)

### --------------------------------------------
### TASK 1: change dt
dt = 0.1 # time step (ms)
### --------------------------------------------

# create a simulator
sim = Simulator()

# create a neuron with input
nrn = MyFirstNeuron(23, 11, 'my 1st neuron!')
exc_input = np.random.randint(0, T, size=10)
inh_input = np.random.randint(0, T, size=10)

nrn.set_inh_input(inh_input)
nrn.set_exc_input(exc_input)

### --------------------------------------------
### TASK 2: create ANOTHER neuron with external input
nrn_with_ext_input = MyFirstNeuron(23, 11, 'my 1st neuron with external input')
### EDIT BELOW >>>
# ...
### <<< EDIT ABOVE
### --------------------------------------------

# adding all neurons we want to update
sim.add_neuron(nrn)
sim.add_neuron(nrn_with_ext_input)

# run a simulation of a neuron
sim.run(T, dt)

# visualise
import matplotlib.pyplot as plt

# create a figure with an axis for plotting
fig, ax = plt.subplots()

# plot voltage over iterations
names = []
for n in sim.neurons:
    ax.plot(sim.t_list, n.V_t)
    names.append(n.name)

# this is a *list comprehension*
names = [n.name for n in sim.neurons]
ax.legend(names, frameon=False)

# add some axis labels
ax.set_ylabel("Voltage [$V_m$] (mV)")
ax.set_xlabel("Time (ms)")

# don't forget a title
ax.set_title("How does external input affect voltage?")

# make the borders prettier
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

plt.show()

**Task 3**

Instead of the **analytical** solution (can compute any time `t`) for `g_syn`, implement the **numerical** solution (need to know `t-1` to calculate `t`).

$\begin{aligned}
\frac{dg_{syn}}{dt} &= -\frac{g_{syn}}{\tau_{syn}} \\
g_{syn} &= g_{syn} + g_{syn_{\rm{max}}}   (\textrm{when } t = t_{s})
\end{aligned}$

Further questions
1. What are the advantages + disadvantages between the 2 solutions?
1. Derive the numerical solution from the analytical solution (ignore the $\sum$)
1. Integrate the numerical solution to get the analytical solution

## The Leaky Integrate-and-Fire Neuron 🔥

Thus far, we have implemented the change in voltage given inputs, but there's no concept of an action potential going on!

The simplest approach is to define a *threshold* $V_{thresh}$ for the neuron that once reached at time $t_i$

1. $V_m$ goes to `0` (or `20`) at $t_{i+1}$
1. After hyperpolarises to $V_{reset}$  at $t_{i+2}$

Because we want to add extra functionality, we'll use this opportunity to introduce another 2 (of the 3) tenets of programming. **Inheritance** and **Polymorphism** (the other tenet was **encapsulation**).

### Inheritance 🧬



We already have a bunch of logic in `MyFirstNeuron` that has the same applicability to a leaky integrate-and-fire (LIF) neuron (setting initial values in `__init__`, adding inputs, summing synaptic inputs, and re-initialising the neuron's voltage). 

We can *inherit* this logic using 

```
class <NewClass>(<ClassToInheritFrom>):
```

> note that previously we *inherited from object*. 

All methods that are in `<ClassToInheritFrom>` (e.g. `MyFirstNeuron`) are accessible in this new class!

```
class LIFNeuron(MyFirstNeuron):
```



But what if we want to *change* a method.

For example, the `step` method in `MyFirstNeuron` doesn't have the logic to handle our (artificial) action potentials  -the "fire" 🔥 in "integrate-and-fire". It only has the sub-threshold "integrate" part. 

> Q: Where does the "leaky" come from?


### Polymorphism 𐩯

To change this, we create a `step` method in this new class with the new logic we want. Using the same name to implement new functionality is known as **polymorphism**! Python determines which `step` method you want by the object that calls it (i.e. is it a `MyFirstNeuron` or `LIFNeuron`).

In fact, the way we have implemented things, we do need to explicitly *repeat* some logic. We could go back to `MyFirstNeuron` and **refactor** `step` to make things smoother by re-using the same logic. But we're only repeating logic once so we'll accept this for now. If this happens again, go back and optimise to reduce sources of error and the number of places you would need to alter something.

```
def step(self, t, dt):
  V_t = self.V_t[-1] # get last value
  if V_t == 0:
    # neuron has spiked at previous time step
    V_t = self.V_rest
  elif V_t >= V_thresh: # what if you change the order of the if statements?
    # voltage has reached threshold, spike!
    V_t = 0
  else:
    # this is repeated from MyFirstNeuron
    I_leak = (V_t - self.V_rest)/self.R_m
    dV = (-I_leak - self.__I_syn(t, V_t) + self.I_e/self.A) * dt/self.C_m
    V_t = V_t + dV
  self.V_t.append(V_t)
```

### Polymorphism with Inheritance 𐩯xx

We also have new parameters, `V_thresh` and `V_reset`, we want to introduce that only needs to be defined once when we create (**instantiate**) our neuron. 

We overwrite `__init__` by  creating a method with the same name, but then we **call** the **parent** (`MyFirstNeuron`) using the super special `super()` function, along with the method we want: `super().__init__()`. 

Because our parent method has parameters it accepts, we need to pass these along the hierarchy. We take in arbitrary *arguments* (e.g. `radius`) and *keyword arguments* (e.g. `R_m=1`) that other methods want with `*args` and `**kwargs`, represpectively. 

> We can explicitly accept arguments and pass them on too, e.g. maybe we want a different default value!

### Another thing: [Logging](https://docs.python.org/3/library/logging.html)  🪵
We have been using `print` to display text, but a better (more informative and robust) way is using the `logging` module.

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s %(name)-15s %(levelname)-8s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)

logger = logging.getLogger('MyLogger')

logger.setLevel(logging.INFO)
 
logger.debug("Only show when debugging")
logger.info("replace print statements with this instead")
logger.warning("this displays a warning")
logger.error("there was an error")


### Implementation of Leaky Integrate-and-Fire Neuron

In [ ]:
class LIFNeuron(MyFirstNeuron):
    """
    Leaky integrate-and-fire neuron that inherits from MyFirstNeuron.
    """
    def __init__(self, *args, 
               name="LIF", # replace default
               V_thresh=-55, V_reset=-68, # new arguments
               **kwargs): # other keyword arguments
        logging.debug(f"other arguments passed = {args}")
        logging.debug(f"other keyword arguments passed = {kwargs}")
        super().__init__(*args, name=name, **kwargs)
        self.V_thresh = V_thresh
        self.V_reset = V_reset

    def step(self, t, dt):
        V_t = self.V_t[-1] # get last value
        if V_t == 0:
            # neuron has spiked at previous time step
            V_t = self.V_reset
        elif V_t >= self.V_thresh: # what if you change the order of the if statements?
            # voltage has reached threshold, spike!
            V_t = 0
        else:
            # this is repeated from MyFirstNeuron, 
            # but we could also call super().step(t, dt) with some consideration
            # (the parent step method also appends to V_t list)
            I_leak = (V_t - self.V_rest)/self.R_m
            dV = (-I_leak - self._I_syn(t, V_t) + self.I_e*1e-5/self.A) * dt/self.C_m
            V_t = V_t + dV
        self.V_t.append(V_t)

Create a shortcut for visualisations

In [ ]:
# visualise
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.cbook import flatten

def plot_v(sim, attrs=None, title="", **kwargs):
    # note that list parameters should have defaults as None in the method and 
    #   defined in the body, even if it's empty
    if attrs is None:
        attrs = []

    # create a figure with an axis for plotting
    fig, axs = plt.subplots(nrows=1+len(attrs), squeeze=False, sharex=True,
                          gridspec_kw={'height_ratios':[5]+[1]*len(attrs)})
    ax = axs[0, 0]
  # plot voltage over iterations
    for neuron in sim.neurons:
        ax.plot(sim.t_list, neuron.V_t, **kwargs)
        for i, attr in enumerate(attrs):
            try:
                axs[i+1,0].plot(sim.t_list, getattr(neuron, attr))
            except AttributeError:
                logger.debug(f"attribute {attr} did not exist in {neuron.name}. Continuing...")
                logger.debug(f"""do this next time `val = getattr(neuron, attr, None)`
                              [see `help(getattr)`]""")

    names = [n.name for n in sim.neurons]
    ax.legend(names, loc='upper left', bbox_to_anchor=(1,1), frameon=False)

    # add some axis labels
    ax.set_ylabel("$V_m$ (mV)") # <-- math symbols in-between $ $
    axs[-1, -1].set_xlabel("Time (ms)")

    # don't forget a title
    ax.set_title(title)

    # make the borders prettier
    for ax in flatten(axs):
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)

    return fig, axs

In [ ]:
import numpy as np
# create a simulator
sim = Simulator()
sim.clean()

T = 1000 # duration (ms)
dt = 0.1 # time step (ms)

# create a neuron with input
lif_nrn = LIFNeuron(23, 11)
lif_nrn.set_inh_input(np.random.randint(0, T, size=10))
lif_nrn.set_exc_input(np.random.randint(0, T, size=10))

sim.add_neuron(lif_nrn)

# run a simulation of a neuron
sim.run(T, dt)

plot_v(sim, title="Let there be spikes!")
print(f"there were {np.sum(np.array(lif_nrn.V_t)>=0)} spikes")

### How does *external input current* affect the *firing rate* of a neuron?

In [ ]:
sim = Simulator()
# get rid of previous neurons
sim.clean()

# create neurons each with their own external current value
for current in np.arange(0,0.5,0.05):
    nrn = LIFNeuron(23, 11, name=f"{current:.2f}")
    nrn.set_external_input(current)
    sim.add_neuron(nrn)
sim.run(T, dt)

In [ ]:
#@title
fig, ax = plot_v(sim, title="Let there be spikes!", alpha=0.5, lw=1)
ax[0,0].get_legend().set_title("External\nCurrent (μA)")

In [ ]:
# convert Voltage for each neuron to an array
v = np.empty(shape=(len(sim.t_list), len(sim.neurons)))
for i,nrn in enumerate(sim.neurons):
    v[:,i] = nrn.V_t
# calculate the instantaneous firing rate
ifr = np.sum(v>=0,axis=0)/(sim.t_list[-1]/1000)

In [ ]:
fig, ax = plt.subplots()
ax.plot([float(nrn.name) for nrn in sim.neurons], ifr)
ax.set_xlabel("Current (μA)")
ax.set_ylabel("Firing rate (Hz)")
ax.set_title("I-F 'curve'")

### **Homework/Project 2**: How does _**excitatory** and **inhibitory** input_ affect the *firing rate* of a neuron?

Similar to above for external current, determine
- how does excitatory drive affect firing rate? 
  - you should produce an input-output (I-O) curve where excitation is on the x-axis and firing rate is on the y-axis

- how does inhibition affect this I-O relationship?

- how does timing of the excitatory + inhibitory inputs affect the relationship? 
  - what if they both have the same input frequency ("Balanced input")
  - what if they have the same frequency, but are offset?
  - what does noise add or take away from this I-O function?

- create a circuit with 2 excitatory neurons and 1 inhibitory neuron. Connect them to each other. \
  *hint when one fires, add a spike time to its connected pair*
  - Increase the number of neurons (try keep a 80:20 percentage of excitatory: inhibitory neurons).
  - Implement Hebbian learning